In [49]:
import glob, os

def rename(dir, pattern, titlePattern):
    for pathAndFilename in glob.iglob(os.path.join(dir, pattern)):
        title, ext = os.path.splitext(os.path.basename(pathAndFilename))
        os.rename(pathAndFilename, os.path.join(dir, titlePattern % title + ext))

In [51]:
rename(r'data/virus_total/',r'*.*',r'*.txt')

In [61]:
## add extension .txt

# root = os.getcwd()
root = 'data/virus_total/'
print(root)
for file in os.listdir('data/virus_total/'):
#     print(file)
    if not os.path.isfile(file):
        continue

#     head, tail = os.path.splitext(file)
#     print(head,tail)
#     if not tail:
    src = os.path.join(root , file)
    dst = os.path.join(root , file + '.txt')

    if not os.path.exists(dst): # check if the file doesn't exist
        os.rename(src, dust)

data/virus_total/


In [62]:
import os,sys
folder = 'data/virus_total/'
for filename in os.listdir(folder):
    infilename = os.path.join(folder,filename)
    if not os.path.isfile(infilename): continue
    oldbase = os.path.splitext(filename)
    newname = infilename.replace(r'*', r'*.txt')
    output = os.rename(infilename, newname)

## 以上沒路用
### 以下修改副檔名

In [65]:
## add extension .txt
for filename in os.listdir('data/virus_total/'):
    base_file, ext = os.path.splitext(filename)
    if ext != ".txt":
        os.rename('data/virus_total/'+filename, 'data/virus_total/'+filename + ".txt")

***

## to do all family dir in main Automatically 
### plz set parameters in __main__ function
#### by leoqaz12 @20180328

In [1]:
def pre_search(in_directory,in_hooklog_directory,in_first_seen=True):
# MIKE: 20170822, hack for TXT (VT report) or hooklog
#     run_directory = in_hooklog_directory if in_hooklog_directory != None else in_directory

    # iter the directory
    file_list = next(os.walk(in_hooklog_directory))[2]
#     print(file_list)
    hash_set = set(t.split('.')[0].split("_")[0] for t in file_list)
    ext = file_list[0].split('.')[-1].lower() 

    print("%d files" % len(file_list))
    print("%d hashes" % len(hash_set))
#     print(ext)
#     print("save to", out_csvfile)
#     print("save to", out_wn_csvfile)

    # find all anti-virus engines and corresponding detection strings

    av_set = set() # set of all anti-virus engines
    csv_dict = dict()
    unknown = []

    for h in hash_set:
        # open txt file and load it as json
        try:
            with open(os.path.join(in_directory, h + '.txt')) as txt_file:    
                json_report = json.load(txt_file)
#                 print(json_report)
        except:
            unknown.append(os.path.join(in_directory, h ).split('/')[-1])

        # create a dictionary _dict = {engine: "detection_name"}
        try:
            _dict = dict()
            for engine in json_report['scans'].keys(): 
                scan_result = json_report['scans'].get(engine)
                if scan_result.get("detected") == True:
                    result = scan_result.get("result").encode('ascii', 'ignore')
                    result = result.decode("ascii").replace(',', '') # special replacement for csv
                    av_set.add(engine)

                    _dict[engine] = result

            # if you don't need first_seen, set in_first_seen as Fasle
            if in_first_seen:
                _dict["first_seen"] = json_report['first_seen']

            # attach this dictioary to csv_dict = {hash: _dict}
            csv_dict[h] = _dict
        except Exception as e:
            print('!!!!!!!!!!!!!!!!!',str(e),'!!!!!!!!!!!!!!!!!')
            pass

    print("沒有vtreport的數量 / main全部的hash數目 :",len(unknown), "/", len(hash_set))
    # unknown

    df = pd.DataFrame(csv_dict).T
    return df
# You can print the df here
# df.head()
#df['AVG']

In [2]:
def split_T_V(df,in_hooklog_directory,out_train,out_valid,split_rate=0.9):    
    sorts = df.filter(['first_seen'])
    sorts = sorts.sort_values(['first_seen'],ascending=[1])
    sorts = sorts.reset_index()
    sorts.columns=['hls','time']
    sort = sorts.drop(columns='time')
#     sort

    split_T = int(len(sorts)*split_rate)
    sort_T , sort_V = sort[:split_T], sort[split_T:] 
    sort_V.reset_index(inplace=True)

    if not os.path.isdir(out_train):
        os.makedirs(out_train)
    for hls in sort_T['hls']:
        pattern_ = in_hooklog_directory + str(hls) + "*"
#         print(pattern_)
        for v in glob.glob(pattern_):
            whole_name = v
            shutil.copy(whole_name, os.path.join(out_train, whole_name.split('/')[-1]))
    print(os.path.join(out_train))
    if not os.path.isdir(out_valid):
        os.makedirs(out_valid)
    for hls in sort_V['hls']:
        pattern_ = in_hooklog_directory + str(hls) + "*"
        whole_name = glob.glob(pattern_)[0]
        for v in glob.glob(pattern_):
            whole_name = v
            shutil.copy(whole_name, os.path.join(out_valid, whole_name.split('/')[-1]))
    print(os.path.join(out_valid))
#     return

In [3]:
#for many generation(or below cell to choose one)
def main(in_directory,in_hooklog_directory,out_train,out_valid,split_rate):
    
    df = pre_search(in_directory,in_hooklog_directory)
    try:
        split_T_V(df,in_hooklog_directory,out_train,out_valid,split_rate)
    except KeyError as e:
        print("!!!!!!!!","KeyError: ",str(e),'!!!!!!!!!!!')
        pass
    


    return

if __name__ == '__main__':
    import os,sys
    import json
    import pandas as pd
    from numpy import nan as NaN
    import shutil
    import glob
#     in_tag = "autoit/main/"

    #PARAMETERS
    in_directory = "./data/VTReport/" # virus total private key report(包含firstseen)，底下是全部的 .txt
    in_hooklog_directorys = "./data/top10_5up/" # hooklog的根目錄，下面有"家族family name"名稱，在下面有"代別generation"名稱
    split_rate = 0.9 #多少%要當訓練資料
    out_trains = 'data/train_top10_5up/' #輸出訓練hooklogs
    out_valids = 'data/test_top10_5up/'  #輸出測試hooklogs

    
    for family in next(os.walk(in_hooklog_directorys))[1]:
#         print(family)
        for gen in next(os.walk(in_hooklog_directorys+family))[1]:  #gen
            print("==========",str(family)+"/"+str(gen)+"/","==========")
            in_hooklog_directory = in_hooklog_directorys+family+"/"+gen+"/"
            out_train = out_trains+family+"/"+gen+"/"
            out_valid = out_valids + family+"/"+gen+"/"
            main(in_directory,in_hooklog_directory,out_train,out_valid,split_rate=0.9)

#     in_first_seen = True


In [4]:
## for only one dir (without generation)

def main(in_directory,in_hooklog_directory,out_train,out_valid,split_rate):
    
    df = pre_search(in_directory,in_hooklog_directory)
    try:
        split_T_V(df,in_hooklog_directory,out_train,out_valid,split_rate)
    except KeyError as e:
        print("!!!!!!!!","KeyError: ",str(e),'!!!!!!!!!!!')
        pass
    


    return

if __name__ == '__main__':
    import os
    import json
    import pandas as pd
    from numpy import nan as NaN
    import shutil
    import glob
#     in_tag = "autoit/main/"

    #PARAMETERS
    in_directory = "./data/VTReport/" # virus total private key report(包含firstseen)，底下是全部的 .txt
    in_hooklog_directorys = "./data/top10_5up/" # hooklog的根目錄，下面有"家族family name"名稱，在下面有"代別generation"名稱
    split_rate = 0.9 #多少%要當訓練資料
    out_trains = 'data/train_top10_5up/' #輸出訓練hooklogs
    out_valids = 'data/test_top10_5up/'  #輸出測試hooklogs

    
    for family in next(os.walk(in_hooklog_directorys))[1]:
        print("==========",str(family)+"/","==========")
        in_hooklog_directory = in_hooklog_directorys+family+"/"
        out_train = out_trains+family+"/"
        out_valid = out_valids + family+"/"
        main(in_directory,in_hooklog_directory,out_train,out_valid,split_rate=0.9)

#     in_first_seen = True


========== ymeta/ ==========
7 files
3 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 3
data/train_top10_5up/ymeta/
data/test_top10_5up/ymeta/
========== black/ ==========
6 files
5 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 5
data/train_top10_5up/black/
data/test_top10_5up/black/
========== gbot/ ==========
5 files
4 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 4
data/train_top10_5up/gbot/
data/test_top10_5up/gbot/
========== linkury/ ==========
10 files
10 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 10
data/train_top10_5up/linkury/
data/test_top10_5up/linkury/
========== krap/ ==========
34 files
26 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 26
data/train_top10_5up/krap/
data/test_top10_5up/krap/
========== mcool/ ==========
70 files
53 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 53
data/train_top10_5up/mcool/
data/test_top10_5up/mcool/
========== casonline/ ==========
8 files
7 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 7
data/train_top10_5up/casonline/
data/test_top10_5up/casonline/
======

37 files
24 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 24
data/train_top10_5up/bladabindi/
data/test_top10_5up/bladabindi/
========== sytro/ ==========
289 files
192 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 192
data/train_top10_5up/sytro/
data/test_top10_5up/sytro/
========== vbobfus/ ==========
32 files
21 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 21
data/train_top10_5up/vbobfus/
data/test_top10_5up/vbobfus/
========== stagol/ ==========
5 files
2 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 2
data/train_top10_5up/stagol/
data/test_top10_5up/stagol/
========== smssend/ ==========
7 files
5 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 5
data/train_top10_5up/smssend/
data/test_top10_5up/smssend/
========== muldrop4/ ==========
5 files
5 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 5
data/train_top10_5up/muldrop4/
data/test_top10_5up/muldrop4/
========== magania/ ==========
7 files
4 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 4
data/train_top10_5up/magania/
data/test_top10_5up/magania/
====

16 files
6 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 6
data/train_top10_5up/netfilter/
data/test_top10_5up/netfilter/
========== dorifel/ ==========
5 files
2 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 2
data/train_top10_5up/dorifel/
data/test_top10_5up/dorifel/
========== lipler/ ==========
5 files
2 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 2
data/train_top10_5up/lipler/
data/test_top10_5up/lipler/
========== s-f8e7a1cd/ ==========
6 files
2 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 2
data/train_top10_5up/s-f8e7a1cd/
data/test_top10_5up/s-f8e7a1cd/
========== hoax/ ==========
12 files
12 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 12
data/train_top10_5up/hoax/
data/test_top10_5up/hoax/
========== otwycal/ ==========
15 files
12 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 12
data/train_top10_5up/otwycal/
data/test_top10_5up/otwycal/
========== kazy/ ==========
9 files
7 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 7
data/train_top10_5up/kazy/
data/test_top10_5up/kazy/
========== juched

沒有vtreport的數量 / main全部的hash數目 : 0 / 33
data/train_top10_5up/simda/
data/test_top10_5up/simda/
========== waski/ ==========
36 files
15 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 15
data/train_top10_5up/waski/
data/test_top10_5up/waski/
========== downloader6/ ==========
5 files
4 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 4
data/train_top10_5up/downloader6/
data/test_top10_5up/downloader6/
========== gamarue/ ==========
5 files
2 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 2
data/train_top10_5up/gamarue/
data/test_top10_5up/gamarue/
========== poison/ ==========
7 files
5 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 5
data/train_top10_5up/poison/
data/test_top10_5up/poison/
========== rozena/ ==========
10 files
5 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 5
data/train_top10_5up/rozena/
data/test_top10_5up/rozena/
========== ntrootkit/ ==========
5 files
1 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 1
data/train_top10_5up/ntrootkit/
data/test_top10_5up/ntrootkit/
========== domaiq/ =======

data/train_top10_5up/dealply/
data/test_top10_5up/dealply/
========== soft32downloader/ ==========
98 files
78 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 78
data/train_top10_5up/soft32downloader/
data/test_top10_5up/soft32downloader/
========== ircbot/ ==========
69 files
44 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 44
data/train_top10_5up/ircbot/
data/test_top10_5up/ircbot/
========== browsefox/ ==========
254 files
144 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 144
data/train_top10_5up/browsefox/
data/test_top10_5up/browsefox/
========== katusha/ ==========
7 files
6 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 6
data/train_top10_5up/katusha/
data/test_top10_5up/katusha/
========== deltree/ ==========
15 files
1 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 1
data/train_top10_5up/deltree/
data/test_top10_5up/deltree/
========== pirminay/ ==========
5 files
4 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 4
data/train_top10_5up/pirminay/
data/test_top10_5up/pirminay/
========== gamania/ =====

data/train_top10_5up/downloadware/
data/test_top10_5up/downloadware/
========== indus/ ==========
5 files
3 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 3
data/train_top10_5up/indus/
data/test_top10_5up/indus/
========== qvod/ ==========
24 files
12 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 12
data/train_top10_5up/qvod/
data/test_top10_5up/qvod/
========== funweb-k/ ==========
6 files
5 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 5
data/train_top10_5up/funweb-k/
data/test_top10_5up/funweb-k/
========== downloader14/ ==========
8 files
3 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 3
data/train_top10_5up/downloader14/
data/test_top10_5up/downloader14/
========== downloader12/ ==========
8 files
4 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 4
data/train_top10_5up/downloader12/
data/test_top10_5up/downloader12/
========== expiro/ ==========
324 files
174 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 174
data/train_top10_5up/expiro/
data/test_top10_5up/expiro/
========== siggen3/ ==========
9 file

data/train_top10_5up/flood/
data/test_top10_5up/flood/
========== hllp/ ==========
30 files
9 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 9
data/train_top10_5up/hllp/
data/test_top10_5up/hllp/
========== swisyn/ ==========
9 files
5 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 5
data/train_top10_5up/swisyn/
data/test_top10_5up/swisyn/
========== picsys/ ==========
76 files
46 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 46
data/train_top10_5up/picsys/
data/test_top10_5up/picsys/
========== core/ ==========
45 files
23 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 23
data/train_top10_5up/core/
data/test_top10_5up/core/
========== download/ ==========
9 files
3 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 3
data/train_top10_5up/download/
data/test_top10_5up/download/
========== darkkomet/ ==========
15 files
8 hashes
沒有vtreport的數量 / main全部的hash數目 : 0 / 8
data/train_top10_5up/darkkomet/
data/test_top10_5up/darkkomet/
========== unclassifiedmalware/ ==========
299 files
202 hashes
沒有vtreport的數量 / ma

***

***

## to do Manually

### parseVTRep2CSV Example

Given a directory having virustotal's json report
Output a CSV table of detection name (column: anti-virus engine; row: malware)

In [65]:
import os
import json
import pandas as pd
from numpy import nan as NaN

In [118]:
# input
in_tag = "lydra"
in_directory = "./data/VTReport/" # your VTreport dir
in_hooklog_directory = "./data/tracelogs_analysis_temu20/"+in_tag+"/main/" # a hooklog dir, or None

in_first_seen = True
in_save_to_subdir_firstseen = True

# output
out_tag = in_tag

# outfiles
out_csvfile = 'output/VTRepo_'+ out_tag + '.csv' # original
out_wn_csvfile = 'output/VTRepo_wn_'+ out_tag + '.csv' # winnowed

In [ ]:
# file_list

In [119]:
# MIKE: 20170822, hack for TXT (VT report) or hooklog
run_directory = in_hooklog_directory if in_hooklog_directory != None else in_directory

# iter the directory
file_list = next(os.walk(run_directory))[2]
hash_set = set(t.split('.')[0].split("_")[0] for t in file_list)
ext = file_list[0].split('.')[-1].lower() 

print("%d files" % len(file_list))
print("%d hashes" % len(hash_set))
print(ext)
print("save to", out_csvfile)
print("save to", out_wn_csvfile)

31 files
28 hashes
hooklog
save to output/VTRepo_lydra.csv
save to output/VTRepo_wn_lydra.csv


In [120]:
# find all anti-virus engines and corresponding detection strings

av_set = set() # set of all anti-virus engines
csv_dict = dict()
unknown = []

for h in hash_set:
    # open txt file and load it as json
    try:
        with open(os.path.join(in_directory, h + '.txt')) as txt_file:    
            json_report = json.load(txt_file)
    except:
        unknown.append(os.path.join(in_directory, h ).split('/')[-1])
        
    # create a dictionary _dict = {engine: "detection_name"}
    _dict = dict()
    for engine in json_report['scans'].keys(): 
        scan_result = json_report['scans'].get(engine)
        if scan_result.get("detected") == True:
            result = scan_result.get("result").encode('ascii', 'ignore')
            result = result.decode("ascii").replace(',', '') # special replacement for csv
            av_set.add(engine)
            
            _dict[engine] = result
    
    # if you don't need first_seen, set in_first_seen as Fasle
    if in_first_seen:
        _dict["first_seen"] = json_report['first_seen']
    
    # attach this dictioary to csv_dict = {hash: _dict}
    csv_dict[h] = _dict

In [121]:
print("沒有vtreport的/main全部的hash數目 :",len(unknown), "/", len(hash_set))
unknown

沒有vtreport的/main全部的hash數目 : 0 / 28


[]

In [122]:
df = pd.DataFrame(csv_dict).T

In [105]:
# You can print the df here
df.head()
#df['AVG']

,ALYac,AVG,AVware,Ad-Aware,AegisLab,Agnitum,AhnLab-V3,AntiVir,Antiy-AVL,Arcabit,...,VIPRE,ViRobot,Webroot,Yandex,Zillya,ZoneAlarm,Zoner,eSafe,first_seen,nProtect
00abd45f619f660ad8694f8df91cca983b1a5b09da38cb124609d7bc315e74e9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR/Spy.668547.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-12 09:23:46,NaN
00b268a868f5a26a0ab12c9edb536029f32d4f84497a2eccc4c9eb01add69cd7,NaN,Worm/Autoit.ADEC,NaN,NaN,NaN,Trojan.Malagent.CKG,Win32/Hakaglan.worm.462264,TR/Dropper.Gen,NaN,NaN,...,Trojan.Win32.AutoIT.gen (v),NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-05 18:06:49,Worm/W32.Sohanad.529920
00b4957b88197b4b4c2a6066b698f4f8e5bedcc6a7a2400cef50c4ead4ab313d,NaN,Autoit.ES,NaN,NaN,NaN,NaN,Win-Trojan/Autoit.305824,TR/Rogue.JH.7554630,NaN,NaN,...,Trojan.Win32.Generic!BT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-04 23:36:34,Trojan.Generic.7554630
00b5dc8fb49512c55773848ef0aea80c26623c267ed6f297d0db71debd0aefb3,NaN,Autoit.ES,NaN,NaN,NaN,NaN,Win-Trojan/Autoit.305824,TR/Rogue.JH.7554630,NaN,NaN,...,Trojan.Win32.Generic!BT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-04 23:25:19,Trojan.Generic.7554630
00b8136d28e889f8506e6d73f651e5254ea3b796bbf92d27e926bd2eff248ddb,NaN,Dropper.Generic3.XJQ,NaN,NaN,NaN,Worm.Autoit.SR,Trojan/Win32.CSon,TR/Dropper.Gen,Trojan/Win32.Autoit.gen,NaN,...,Worm.Win32.AutoIt.tq (v),Worm.Win32.A.IM-Sohanad.278196,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-01 23:45:49,Trojan.Generic.6485445


## Save files to new directory based on first-seen year split_rate

要先跑完winnowing以前的cell

In [123]:
split_rate = 0.9
out_train = 'data/train/'+in_tag
out_valid = 'data/test/'+in_tag

In [124]:
# df = pd.read_json('data/VTReport/09238b81c2f66d24305957d6f82520627a3ea6358db0a3b7cc4d10d2c272f073.txt')
# df
# json_report = json.load('data/VTReport/09238b81c2f66d24305957d6f82520627a3ea6358db0a3b7cc4d10d2c272f073.txt')
# json_report
# with open(os.path.join(in_directory, h + '.txt')) as txt_file:    
#     json_report = json.load(txt_file)
# csv_dict
import shutil
import glob
sorts = df.filter(['first_seen'])
sorts = sorts.sort_values(['first_seen'],ascending=[1])
sorts = sorts.reset_index()
sorts.columns=['hls','time']
sort = sorts.drop(columns='time')
sort

,hls
0,01bd862f71708be4f1d31f94a3665632162722a2f5bbb5...
1,01ad61d5f02eb6e4a8f26e65324207d9dfe83d7233109e...
2,f49ad6d63d39dd4406d70169329d8454a63549dc19e342...
3,e757bea3d72a00e9ae28ad4c58b47df8898660124a5c42...
4,0999697353d50372bdaad0de7efaea15db8033c36be69e...
5,d5960cd340de55c24faf62b47b58df82d8a40a741cc501...
6,71362ec6f86ca2c1cf9f0ad828ff86968e1c4f7a020270...
7,9f9bce28b30268e34ae747958303077a80f62521a510b8...
8,3ddf3290b4e2b795d005d6703891f667e13bb13ebb938b...
9,82c29a4df668d6203176f0f1482cf6f6075c5543844b17...


In [125]:
vv=[]
if not os.path.isdir(out_train):
    os.makedirs(out_train)
for hls in sort_T['hls']:
    pattern_ = in_hooklog_directory + str(hls) + "*"
    for v in glob.glob(pattern_):
        whole_name = v
        vv.append(whole_name)
        shutil.copy(whole_name, os.path.join(out_train, whole_name.split('/')[-1]))
print(os.path.join(out_train))
if not os.path.isdir(out_valid):
    os.makedirs(out_valid)
for hls in sort_V['hls']:
    pattern_ = in_hooklog_directory + str(hls) + "*"
    whole_name = glob.glob(pattern_)[0]
    shutil.copy(whole_name, os.path.join(out_valid,whole_name.split('/')[-1]))
print(os.path.join(out_valid))

data/train/lydra


IndexError: list index out of range

In [39]:
# pattern。

NameError: name 'pattern' is not defined

In [81]:
for v in glob.glob(pattern_):
    print(v)

./data/tracelogs_analysis_temu20/autoit/main/7444a1e9bead7c4cd14c375282aa36095776b1261ef2ecaf2af8ca081e9c3fbd_3304.trace.hooklog
./data/tracelogs_analysis_temu20/autoit/main/7444a1e9bead7c4cd14c375282aa36095776b1261ef2ecaf2af8ca081e9c3fbd_3101.trace.hooklog


In [54]:
def split_str(strs):
    return strs.split('\"')[3]

In [ ]:
df = pd.read_table('data/III_malware_10000(gamma dataset)/9ffbc2e30584ca6b0dfb60e304f14e8810c62d4c4db8befeab2472d3e015de9c.txt',header=None)


In [48]:
df2 = df[df[0].str.contains('md5|sha')].reset_index()

In [55]:
s2 = df2[0].apply(split_str)

In [57]:
for v in s2:
    print(v)

a469192ac409198a7695e0790154680c2efa8c2f2c4238f2a465de66756c9a14
4a2bffc860c95e2d9c840770f9de7f2a0518681016e607b7bd163a382dd45283
6eb54801f91b6d8effccbfaefe6b2d7705a274a75940e6226e24e0d4ec58c396
a0c9d012e2bf6b2fe05c2d97cb5594d97cf2f539e97935c12abd7a3562f4d9bf
1e11948157af5ffaa86a4282c697dff2
b9d7a6482c3e6bad6756195c366c2e5a4b3b9785
9ffbc2e30584ca6b0dfb60e304f14e8810c62d4c4db8befeab2472d3e015de9c


In [73]:
split_T = int(len(sorts)*split_rate)
sort_T , sort_V = sort[:split_T], sort[split_T:] 
sort_V.reset_index(inplace=True)

In [95]:
# kk = [kk for i in range(60)]
kk=[]
for i in range(67):
    kk.append(i)
# len(sorts)
# len(kk)
s= int(67*0.9)
s_T,s_V = kk[:s],kk[s:]
print(len(s_T),len(s_V))

60 7


In [86]:
len(sort[:split_T]),len(sort[split_T:])

(86, 10)

In [96]:
cpt = sum([len(files) for r, d, files in os.walk("data/tracelogs_analysis_temu20/")])
cpt

5490

In [98]:
cpt1,cpt2 = sum([len(files) for r, d, files in os.walk("data/train/")]),sum([len(files) for r, d, files in os.walk("data/test/")])
cpt1+cpt2

5318

# Verify correctness

In [6]:
N=0
A=[]
for dirpath, dirnames, filenames in os.walk('data/901family_trace_all_param_gen/'):
    N += len(filenames)
    if dirpath.split('/')[-2] != '901family_trace_all_param_gen' and  dirpath.split('/')[-1] == 'main':
        print("Files in", dirpath.split('/')[-2], dirpath.split('/')[-1], len(filenames))
        A.append(len(filenames))

In [36]:
N=0
t=[]
for dirpath, dirnames, filenames in os.walk('data/train_901family/'):  #放進去的資料夾
    N += len(filenames)
    if dirpath.split('/')[-2] != '901family_trace_all_param_gen' and  dirpath.split('/')[-1] == 'main':
        print("Files in", dirpath.split('/')[-2], dirpath.split('/')[-1],len(filenames))
        t.append(len(filenames))

Files in ymeta main 1
Files in rhtl main 2
Files in bkmf main 2
Files in bxmm main 2
Files in black main 1
Files in reconyc main 1
Files in gbot main 2
Files in linkury main 1
Files in browsecx main 2
Files in krap main 1
Files in rodecap main 1
Files in tibs main 1
Files in buwbakv main 1
Files in whiteice main 2
Files in tufik main 2
Files in androidos main 1
Files in nakuru main 1
Files in plush main 1
Files in pcmega main 1
Files in zzjo main 2
Files in games main 1
Files in hebogo main 1
Files in umnfrj main 2
Files in installflash main 1
Files in bfgp main 1
Files in cccs main 3
Files in outbrowse main 14
Files in midie main 2
Files in expirio main 1
Files in hackkms main 1
Files in bocj main 1
Files in klez main 4
Files in jeefo main 3
Files in dwaeqslb main 2
Files in smheist main 1
Files in whenu main 1
Files in rjzwkib main 1
Files in bodegun main 1
Files in wintrim main 1
Files in pykspa main 1
Files in tofsee main 1
Files in swrort main 2
Files in bdgm main 2
Files in pjxh 

Files in bmgfrpuexrhib main 3
Files in cbuf main 1
Files in powbayfuwyib main 1
Files in ganelp main 1
Files in losabel main 2
Files in delf main 12
Files in downloa main 11
Files in html main 1
Files in vtflooder main 5
Files in sirefef main 4
Files in mailru main 1
Files in desktopmedia main 1
Files in optimu main 1
Files in android main 1
Files in installcore main 22
Files in zona main 2
Files in rzbejkdoy main 1
Files in clientconnect main 1
Files in adload main 2
Files in shopper main 1
Files in slime main 1
Files in arqpcaf main 1
Files in papras main 1
Files in qhosts main 1
Files in qqrob main 1
Files in firseria main 12
Files in visualprotect main 1
Files in badur main 2
Files in vague main 1
Files in minggy main 3
Files in usteal main 2
Files in vilsel main 6
Files in nimnul main 16
Files in fraudload main 1
Files in uvuf main 1
Files in msil main 12
Files in explorerhijack main 2
Files in cashon main 1
Files in sivis main 1
Files in klone main 1
Files in cosmicduke main 1
Fi

In [37]:
N=0
v=[]
for dirpath, dirnames, filenames in os.walk('data/test_901family/'):
    N += len(filenames)
    if dirpath.split('/')[-2] != '901family_trace_all_param_gen' and  dirpath.split('/')[-1] == 'main':
        print("Files in", dirpath.split('/')[-2], dirpath.split('/')[-1], len(filenames))
        v.append(len(filenames))

Files in ymeta main 3
Files in rhtl main 4
Files in bkmf main 0
Files in bxmm main 0
Files in black main 3
Files in reconyc main 0
Files in gbot main 1
Files in linkury main 9
Files in browsecx main 0
Files in krap main 3
Files in rodecap main 0
Files in tibs main 1
Files in buwbakv main 0
Files in whiteice main 0
Files in tufik main 1
Files in androidos main 2
Files in nakuru main 0
Files in plush main 0
Files in pcmega main 1
Files in zzjo main 0
Files in games main 1
Files in hebogo main 0
Files in umnfrj main 0
Files in installflash main 0
Files in bfgp main 0
Files in cccs main 0
Files in outbrowse main 130
Files in midie main 4
Files in expirio main 4
Files in hackkms main 1
Files in bocj main 2
Files in klez main 5
Files in jeefo main 8
Files in dwaeqslb main 0
Files in smheist main 1
Files in whenu main 0
Files in rjzwkib main 0
Files in bodegun main 0
Files in wintrim main 0
Files in pykspa main 1
Files in tofsee main 0
Files in swrort main 5
Files in bdgm main 0
Files in pjxh

Files in pirminay main 4
Files in dyvdpk main 0
Files in ruskill main 0
Files in mediamagnet main 0
Files in horst main 5
Files in gamethief main 0
Files in shellini main 0
Files in gslpvdld main 0
Files in ardamax main 1
Files in fasong main 5
Files in epicgames main 0
Files in zlob main 3
Files in qssko main 0
Files in kykymber main 0
Files in kakavex main 0
Files in flashbot main 0
Files in fujacks main 2
Files in cafm main 0
Files in vbclone main 0
Files in bmgfrpuexrhib main 0
Files in cbuf main 0
Files in powbayfuwyib main 0
Files in ganelp main 2
Files in losabel main 0
Files in delf main 73
Files in downloa main 99
Files in html main 2
Files in vtflooder main 27
Files in sirefef main 35
Files in mailru main 0
Files in desktopmedia main 0
Files in optimu main 6
Files in android main 2
Files in installcore main 158
Files in zona main 1
Files in rzbejkdoy main 0
Files in clientconnect main 4
Files in adload main 21
Files in shopper main 0
Files in slime main 0
Files in arqpcaf mai

In [38]:
import numpy as np
v1 = np.array(v)
t1= np.array(t)
kkk = v1+t1
for i,vv in enumerate(kkk):
    print(vv,A[i])

4 4
6 6
2 2
2 2
4 4
1 1
3 3
10 10
2 2
4 4
1 1
2 2
1 1
2 2
3 3
3 3
1 1
1 1
2 2
2 2
2 2
1 1
2 2
1 1
1 1
3 3
144 144
6 6
5 5
2 2
3 3
9 9
11 11
2 2
2 2
1 1
1 1
1 1
1 1
2 2
1 1
7 7
2 2
1 1
15 15
4 4
14 14
1 1
23 23
3 3
1 1
1 1
1 1
2 2
1 1
29 29
83 83
199 199
14 14
6 6
13 13
1 1
1 1
1 1
1 1
27 27
1 1
8 8
4 4
5 5
2 2
24 24
2 2
1 1
15 15
7 7
218 218
1 1
1 1
1 1
45 45
45 45
1 1
9 9
1 1
14 14
20 20
8 8
1 1
3 3
4 4
2 2
2 2
1 1
1 1
1 1
1 1
13 13
2 2
1 1
6 6
2 2
1 1
2 2
1 1
56 56
5 5
1 1
21 21
1 1
1 1
1 1
16 16
7 7
31 31
1 1
1 1
2 2
1 1
5 5
3 3
3 3
2 2
2 2
1 1
1 1
1 1
1 1
23 23
2 2
340 340
3 3
19 19
1 1
307 307
1 1
8 8
1 1
3 3
2 2
2 2
1 1
1 1
1 1
11 11
5 5
1 1
1 1
5 5
1 1
1 1
4 4
1 1
5 5
2 2
27 27
3 3
1 1
7 7
1 1
1 1
1 1
20 20
2 2
1 1
86 86
1 1
11 11
1 1
34 34
30 30
1 1
1 1
1 1
8 8
4 4
1 1
8 8
61 61
2 2
119 119
2 2
3 3
1 1
4 4
1 1
1 1
1 1
1 1
1 1
11 11
12 12
2 2
23 23
1 1
4 4
14 14
23 23
14 14
1 1
2 2
24 24
42 42
2 2
1 1
2 2
166 166
26 26
34 34
4 4
7 7
6 6
1 1
62 62
1 1
4 4
4 4
2 2
1 1
29 29
324 32